# Imports

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import os
import random
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
device

# Data

In [ ]:
from skimage import io, transform
class UTKFaceDataset(Dataset):

    def __init__(self, dataset_path, transform = None):
        self.dataset_path = dataset_path
        self.transform = transform
        images = []
        genders = []
        ages = []
        for i in os.listdir(self.dataset_path)[0:8000]:
            split = i.split('_')
            ages.append(int(split[0]))
            genders.append(int(split[1]))
            images.append(os.path.join(self.dataset_path, i))
        self.df = pd.DataFrame({"Image": images, "Genders": genders, "Ages": ages})

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image = io.imread(self.df.iloc[index]["Image"])
        label = self.df.iloc[index]["Genders"]
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((64,64),
                          antialias = True)
    ]
)

dataset = UTKFaceDataset('../input/UTKFace',transform)

# Pre-processing

In [ ]:
# train test split
indices = [i for i in range(len(dataset))]
train_val_indices, test_indices = train_test_split(indices, test_size = 0.4, random_state = 31)
train_indices, val_indices = train_test_split(train_val_indices, test_size = 0.3, random_state = 31)

In [ ]:

batch_size = 64
train_loader = DataLoader(torch.utils.data.Subset(dataset,train_indices),
                          batch_size=batch_size, shuffle = True)
val_loader = DataLoader(torch.utils.data.Subset(dataset,val_indices),
                        batch_size=batch_size, shuffle = False)
test_loader = DataLoader(torch.utils.data.Subset(dataset,test_indices),
                         batch_size=batch_size, shuffle = False)

In [ ]:
def get_random_images(dataset, N):
    images = []
    labels = []
    image_index = np.random.choice([i for i in range(len(dataset))],
                                    size = N,
                                    replace = False)
    for i in image_index:
      images.append(dataset[i][0])
      labels.append(dataset[i][1])
    return torch.stack(images), labels

def plot_images(images, nrow = 5):
    greed = torchvision.utils.make_grid(
        tensor = images,
        nrow = nrow
    )
    plt.imshow(greed.cpu().permute(1, 2, 0))
    plt.axis('off')
    plt.show()

In [ ]:
print(f"no. of images : {len(dataset)}")
print(f"Size of images : {dataset[0][0].size()}")
images, labels = get_random_images(dataset, 25)
plot_images(images)

# Model

In [ ]:
class DenseLayer(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(DenseLayer, self).__init__()
        self.Dense_Layer = nn.Sequential(
            nn.Conv2d(in_channels,
                      out_channels,
                      kernel_size = 3,
                      stride = 1,
                      padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels) 
        )

    def forward(self,x):        
        out = torch.cat([x, self.Dense_Layer(x)], dim = 1)
        return out

class DenseBlock(nn.Module):

    def __init__(self, channels):    
        super(DenseBlock, self).__init__()
        self.Dense_Layers = nn.Sequential(
            DenseLayer(channels * 1 , channels),
            DenseLayer(channels * 2 , channels),
            DenseLayer(channels * 3 , channels)
        )
        self.Conv_1x1 = nn.Conv2d(channels * 4, channels, 
                                  kernel_size = 1, 
                                  padding = 0)

    def forward(self, x):
        out = self.Dense_Layers(x)
        out = self.Conv_1x1(out)
        return out


class ConvolutionalBlock(nn.Module):

    def __init__(self, in_channels, out_channels):

        super(ConvolutionalBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels,
                              out_channels,
                              kernel_size = 3,
                              stride = 1,
                              padding = 1)
        self.relu = nn.ReLU()
        self.batchnorm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.batchnorm(x)
        return x

class TransposeConvolutionalBlock(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(TransposeConvolutionalBlock, self).__init__()
        self.conv = nn.ConvTranspose2d(in_channels,
                                       out_channels,
                                       kernel_size = 3,
                                       stride = 2,
                                       output_padding = 1,
                                       padding = 1)
        self.relu = nn.ReLU()
        self.batchnorm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.batchnorm(x)
        return x

class Autoencoder(nn.Module):

    def __init__(self):
        super(Autoencoder, self).__init__()

        # Khushal :
        # 3 x 64 x 64
        # 32 x 32 x 32
        # 64 x 16 x 16
        # 128 x 8 x 8

        self.encoder_conv = nn.Sequential(
            ConvolutionalBlock(3, 32),
            nn.MaxPool2d(kernel_size = 2),
            DenseBlock(32),
            ConvolutionalBlock(32, 64),
            nn.MaxPool2d(kernel_size = 2),
            DenseBlock(64),
            ConvolutionalBlock(64, 128),
            nn.MaxPool2d(kernel_size = 2)
        )

        # Khushal :
        # 128 x 8 x 8
        # 64 x 16 x 16
        # 32 x 32 x 32
        # 3 x 64 x 64

        self.decoder_conv = nn.Sequential(
            TransposeConvolutionalBlock(128, 64),
            TransposeConvolutionalBlock(64, 32),
            nn.ConvTranspose2d(32, 3,
                               kernel_size = 3,
                               stride = 2,
                               output_padding = 1,
                               padding = 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder_conv(x)
        x = self.decoder_conv(x)
        return x

model_ae = Autoencoder().to(device)
print(model_ae)

# Training

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    for epoch in range(num_epochs):

        # Training
        model.train()
        running_train_loss = 0.0
        for i, (images, gender_labels) in enumerate(train_loader):
            optimizer.zero_grad()
            images, gender_labels = images.to(device), gender_labels.to(device)
            reconstruction = model(images)
            loss = criterion(reconstruction, images)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()
        epoch_train_loss = running_train_loss / len(train_loader)
        train_losses.append(epoch_train_loss)

        # Validation
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
          for i, (images, gender_labels) in enumerate(val_loader):
                images, gender_labels = images.to(device), gender_labels.to(device)
                reconstruction = model(images)
                loss = criterion(reconstruction, images)
                running_val_loss += loss.item()
        epoch_val_loss = running_val_loss / len(val_loader)
        val_losses.append(epoch_val_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}')
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

criterion = nn.MSELoss()
optimizer = optim.Adam(model_ae.parameters(), lr = 0.001)
train_model(model_ae, train_loader, val_loader, criterion, optimizer, num_epochs = 20)